In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
from mmms import (utils, plot, io)

/fs/lustre/scratch/mali/gw-distributions/gwdistributions/distributions/location/angles/zodiac/iau.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [ ]:
population_config = 
population, pop_params, pop_variates = io.load_population(population_config, verbose=False)